In [15]:
cd ~/code/git/pudl/pudl

/Users/zaneselvans/code/git/pudl/pudl


In [5]:
import numpy as np
import pandas as pd
import pudl
import ferc1
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [6]:
#import eiaf923
%load_ext autoreload
%autoreload 1
%aimport pudl
%aimport ferc1
#%aimport eia923

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
engine = create_engine('postgresql://catalyst@localhost:5432/pudl_sandbox')
pudl.Base.metadata.drop_all(engine)
pudl.Base.metadata.create_all(engine)

In [11]:
Session = sessionmaker(bind=engine)
session = Session()

In [54]:
map_eia923_ferc1_file = '../results/id_mapping/mapping_eia923_ferc1.xlsx'

plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                          converters={'plant_id':int,
                                      'plant_name':str,
                                      'respondent_id_ferc1':int,
                                      'respondent_name_ferc1':str,
                                      'plant_name_ferc1':str,
                                      'plant_id_eia923':int,
                                      'plant_name_eia923':str,
                                      'operator_name_eia923':str,
                                      'operator_id_eia923':int})

util_map  = pd.read_excel(map_eia923_ferc1_file,'utilities_output',
                          converters={'utility_id':int,
                                      'utility_name':str,
                                      'respondent_id_ferc1':int,
                                      'respondent_name_ferc1':str,
                                      'operator_id_eia923':int,
                                      'operator_name_eia923':str})

plants = plant_map[['plant_id','plant_name']]
plants = plants.drop_duplicates('plant_id')

plants_eia923 = plant_map[['plant_id_eia923','plant_name_eia923','plant_id']]
plants_eia923 = plants_eia923.drop_duplicates('plant_id_eia923')

plants_ferc1 = plant_map[['plant_name_ferc1','respondent_id_ferc1','plant_id']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])

utils = util_map[['utility_id','utility_name']]
utils = utils.drop_duplicates('utility_id')

utils_eia923 = util_map[['operator_id_eia923','operator_name_eia923','utility_id']]
utils_eia923 = utils_eia923.drop_duplicates('operator_id_eia923')

utils_ferc1 = util_map[['respondent_id_ferc1','respondent_name_ferc1','utility_id']]
utils_ferc1 = utils_ferc1.drop_duplicates('respondent_id_ferc1')

In [62]:
plants_ferc1[~plants_ferc1.respondent_id_ferc1.isin(utils_ferc1.respondent_id_ferc1)]

,plant_name_ferc1,respondent_id_ferc1,plant_id


In [7]:
pudl.init_db(pudl.Base)

In [34]:
map_eia923_ferc1_file = '../results/id_mapping/mapping_eia923_ferc1.xlsx'

util_map  = pd.read_excel(map_eia923_ferc1_file,'utilities_output',
                              converters={'utility_id':int,
                                          'utility_name':str,
                                          'respondent_id_ferc1':int,
                                          'respondent_name_ferc1':str,
                                          'operator_id_eia923':int,
                                          'operator_name_eia923':str})

utils_eia923 = util_map[['operator_id_eia923','operator_name_eia923','utility_id']]
utils_eia923 = utils_eia923.drop_duplicates('operator_id_eia923')
utils_ferc1 = util_map[['respondent_id_ferc1','respondent_name_ferc1','utility_id']]
utils_ferc1 = utils_ferc1.drop_duplicates('respondent_id_ferc1')

plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                              converters={'plant_id':int,
                                          'plant_name':str,
                                          'respondent_id_ferc1':int,
                                          'respondent_name_ferc1':str,
                                          'plant_name_ferc1':str,
                                          'plant_id_eia923':int,
                                          'plant_name_eia923':str,
                                          'operator_name_eia923':str,
                                          'operator_id_eia923':int})

plants_eia923 = plant_map[['plant_id_eia923','plant_name_eia923','plant_id']]
plants_eia923 = plants_eia923.drop_duplicates('plant_id_eia923')
plants_ferc1 = plant_map[['plant_name_ferc1','respondent_id_ferc1','plant_id']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])
# There should be at most one row in each of these data frames with NaN values
# after we drop_duplicates in each. This is because there will be some plants and
# utilities that only exist in FERC, or only exist in EIA, and while they will have
# PUDL IDs, they may not have FERC/EIA info (and it'll get pulled in as NaN)
for df in [plants_eia923, plants_ferc1, utils_eia923, utils_ferc1]:
    assert(df[pd.isnull(df).any(axis=1)].shape[0]<=1)
    df.dropna(inplace=True)

In [36]:
for p in plants_eia923.itertuples():
    session.add(pudl.PlantEIA923(plant_id      = int(p.plant_id_eia923),
                            plant_name    = p.plant_name_eia923,
                            plant_id_pudl = int(p.plant_id)))

In [37]:
    for p in plants_ferc1.itertuples():
        session.add(pudl.PlantFERC1(respondent_id = int(p.respondent_id_ferc1),
                               plant_name    = p.plant_name_ferc1,
                               plant_id_pudl = int(p.plant_id)))


In [38]:
for u in utils_eia923.itertuples():
    session.add(pudl.UtilityEIA923(operator_id   = int(u.operator_id_eia923),
                                   operator_name = u.operator_name_eia923,
                                   util_id_pudl  = int(u.utility_id)))


In [39]:
for u in utils_ferc1.itertuples():
    session.add(pudl.UtilityFERC1(respondent_id   = int(u.respondent_id_ferc1),
                             respondent_name = u.respondent_name_ferc1,
                             util_id_pudl    = int(u.utility_id)))

In [40]:
utils = util_map[['utility_id','utility_name']]
utils = utils.drop_duplicates('utility_id')
for u in utils.itertuples():
    session.add(pudl.Utility(id = int(u.utility_id), name = u.utility_name))


In [41]:
plants = plant_map[['plant_id','plant_name']]
plants = plants.drop_duplicates('plant_id')
for p in plants.itertuples():
    session.add(pudl.Plant(id = int(p.plant_id), name = p.plant_name))


In [42]:
session.commit()

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "plants_pkey"
DETAIL:  Key (id)=(1) already exists.
 [SQL: 'INSERT INTO plants (id, name) VALUES (%(id)s, %(name)s)'] [parameters: ({'name': 'Dolet Hills', 'id': 1}, {'name': 'Flint Creek', 'id': 2}, {'name': 'Pirkey', 'id': 3}, {'name': '59th Street', 'id': 4}, {'name': '74th Street', 'id': 5}, {'name': 'Allen S King', 'id': 6}, {'name': 'Thousand Springs', 'id': 7}, {'name': 'A B Brown', 'id': 8}  ... displaying 10 of 7580 total bound parameter sets ...  {'name': 'CoServ Solar Station', 'id': 7580}, {'name': 'State-Fuel Level Increment', 'id': 7581})]